# Feature Engineering

This notebook details the feature engineering techniques used in this project and provides code samples for each technique. All methods discussed this notebooks are applied to the data in the `get_feature_engineered_df()` function found in `data/load_data.py`.

In [1]:
import pandas as pd

example_df = pd.read_csv("../data/season_averages/2019_stats.csv")